In [17]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

In [18]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
#Removed 6500 S Kildare Ave property after previous analysis showed there was likely a typo in LandArea
file_path = Path("Sold_Inflation_M1_12.csv")
df_Sold_Inflation = pd.read_csv(file_path)


# Review the DataFrame
df_Sold_Inflation.head()

,Address,City,Zip,SoldDate,Bathrooms,Bedrooms,LivingArea,LandArea,PropPrice,InflationRate,FederalInt,MonthID
0,244 N Delaware St,Chandler,85225,2/19/24,3.0,4,1272,7532.0,275000,3.1,5.33,2
1,923 W Mesquite St,Chandler,85225,2/16/24,2.0,3,1480,7492.0,428000,3.1,5.33,2
2,304 W El Prado Rd,Chandler,85225,2/16/24,2.0,3,1300,8024.0,419000,3.1,5.33,2
3,1412 E Ironwood Dr,Chandler,85225,2/16/24,2.0,3,1308,7823.0,435300,3.1,5.33,2
4,663 E Manor Dr,Chandler,85225,2/16/24,2.0,3,1415,8407.0,494000,3.1,5.33,2


In [19]:
df_Sold_Inflation.dtypes

Address           object
City              object
Zip                int64
SoldDate          object
Bathrooms        float64
Bedrooms           int64
LivingArea         int64
LandArea         float64
PropPrice          int64
InflationRate    float64
FederalInt       float64
MonthID            int64
dtype: object

In [20]:
#convert LandArea reported in acres to sqft (anything less than a value of 50)
df_Sold_Inflation['LandArea'] = df_Sold_Inflation['LandArea'].astype(float)
for index, row in df_Sold_Inflation.iterrows():
    if row['LandArea'] < 50:
        df_Sold_Inflation.at[index, 'LandArea'] *= 43560

df_Sold_Inflation_new=df_Sold_Inflation
df_Sold_Inflation_new

,Address,City,Zip,SoldDate,Bathrooms,Bedrooms,LivingArea,LandArea,PropPrice,InflationRate,FederalInt,MonthID
0,244 N Delaware St,Chandler,85225,2/19/24,3.0,4,1272,7532.0,275000,3.10,5.33,2
1,923 W Mesquite St,Chandler,85225,2/16/24,2.0,3,1480,7492.0,428000,3.10,5.33,2
2,304 W El Prado Rd,Chandler,85225,2/16/24,2.0,3,1300,8024.0,419000,3.10,5.33,2
3,1412 E Ironwood Dr,Chandler,85225,2/16/24,2.0,3,1308,7823.0,435300,3.10,5.33,2
4,663 E Manor Dr,Chandler,85225,2/16/24,2.0,3,1415,8407.0,494000,3.10,5.33,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2144,5469 NW Meadowlands Ter,Portland,97229,1/2/24,3.0,4,2592,6969.6,765000,3.09,5.33,1
2145,1837 NW Caitlin Ter,Portland,97229,12/29/23,3.0,4,2146,8712.0,700000,3.35,5.33,12
2146,11060 NW Cornell Rd,Portland,97229,12/29/23,2.0,3,2048,16117.2,574000,3.35,5.33,12
2147,200 NW 101st Ave,Portland,97229,12/29/23,3.0,3,2520,17859.6,600000,3.35,5.33,12


In [21]:
df_Sold_Inflation_new['Zip'] = df_Sold_Inflation_new['Zip'].astype(str)
df_Sold_Inflation_new

,Address,City,Zip,SoldDate,Bathrooms,Bedrooms,LivingArea,LandArea,PropPrice,InflationRate,FederalInt,MonthID
0,244 N Delaware St,Chandler,85225,2/19/24,3.0,4,1272,7532.0,275000,3.10,5.33,2
1,923 W Mesquite St,Chandler,85225,2/16/24,2.0,3,1480,7492.0,428000,3.10,5.33,2
2,304 W El Prado Rd,Chandler,85225,2/16/24,2.0,3,1300,8024.0,419000,3.10,5.33,2
3,1412 E Ironwood Dr,Chandler,85225,2/16/24,2.0,3,1308,7823.0,435300,3.10,5.33,2
4,663 E Manor Dr,Chandler,85225,2/16/24,2.0,3,1415,8407.0,494000,3.10,5.33,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2144,5469 NW Meadowlands Ter,Portland,97229,1/2/24,3.0,4,2592,6969.6,765000,3.09,5.33,1
2145,1837 NW Caitlin Ter,Portland,97229,12/29/23,3.0,4,2146,8712.0,700000,3.35,5.33,12
2146,11060 NW Cornell Rd,Portland,97229,12/29/23,2.0,3,2048,16117.2,574000,3.35,5.33,12
2147,200 NW 101st Ave,Portland,97229,12/29/23,3.0,3,2520,17859.6,600000,3.35,5.33,12


In [22]:
# Encode (convert to dummy variables) the EnergyType column
df_zip_dummies = pd.get_dummies(df_Sold_Inflation_new["Zip"])
df_month_dummies = pd.get_dummies(df_Sold_Inflation_new["MonthID"])

In [23]:
# Scaling the numeric columns
df_Sold_Inflation_scaled = StandardScaler().fit_transform(df_Sold_Inflation_new[["Bathrooms", "Bedrooms", "LivingArea","LandArea","PropPrice"]])

# Creating a DataFrame with with the scaled data
df_sold_transformed = pd.DataFrame(df_Sold_Inflation_scaled, columns=["Bathrooms", "Bedrooms", "LivingArea","LandArea","PropPrice"])

# # Display sample data
df_sold_transformed

,Bathrooms,Bedrooms,LivingArea,LandArea,PropPrice
0,-0.103040,0.013159,-0.519985,-0.150507,-0.407218
1,-0.545243,-0.615242,-0.447988,-0.151883,-0.382662
2,-0.545243,-0.615242,-0.510293,-0.133586,-0.384107
3,-0.545243,-0.615242,-0.507524,-0.140499,-0.381491
4,-0.545243,-0.615242,-0.470487,-0.120414,-0.372070
...,...,...,...,...,...
2144,-0.103040,0.013159,-0.063082,-0.169849,-0.328577
2145,-0.103040,0.013159,-0.217460,-0.109924,-0.339009
2146,-0.545243,-0.615242,-0.251381,0.144756,-0.359231
2147,-0.103040,-0.615242,-0.088004,0.204681,-0.355058


In [24]:
# Concatenate the df_zip_dummies and the df_month_dummies DataFrames
df_sold_transformed = pd.concat([df_sold_transformed, df_Sold_Inflation_new['Zip']], axis=1)

# Display sample data
df_sold_transformed

,Bathrooms,Bedrooms,LivingArea,LandArea,PropPrice,Zip
0,-0.103040,0.013159,-0.519985,-0.150507,-0.407218,85225
1,-0.545243,-0.615242,-0.447988,-0.151883,-0.382662,85225
2,-0.545243,-0.615242,-0.510293,-0.133586,-0.384107,85225
3,-0.545243,-0.615242,-0.507524,-0.140499,-0.381491,85225
4,-0.545243,-0.615242,-0.470487,-0.120414,-0.372070,85225
...,...,...,...,...,...,...
2144,-0.103040,0.013159,-0.063082,-0.169849,-0.328577,97229
2145,-0.103040,0.013159,-0.217460,-0.109924,-0.339009,97229
2146,-0.545243,-0.615242,-0.251381,0.144756,-0.359231,97229
2147,-0.103040,-0.615242,-0.088004,0.204681,-0.355058,97229


In [25]:
#Split above dataframe by zip
df_Az=df_sold_transformed[df_sold_transformed['Zip']=='85225']
df_Chi=df_sold_transformed[df_sold_transformed['Zip']=='60629']
df_LA=df_sold_transformed[df_sold_transformed['Zip']=='90210']
df_Fl=df_sold_transformed[df_sold_transformed['Zip']=='33186']
df_NY=df_sold_transformed[df_sold_transformed['Zip']=='11368']
df_OR=df_sold_transformed[df_sold_transformed['Zip']=='97229']


In [26]:
#Split original dataframe (without transformations) by zip
df_Az_Orig=df_Sold_Inflation_new[df_Sold_Inflation_new['Zip']=='85225']
df_Chi_Orig=df_Sold_Inflation_new[df_Sold_Inflation_new['Zip']=='60629']
df_LA_Orig=df_Sold_Inflation_new[df_Sold_Inflation_new['Zip']=='90210']
df_Fl_Orig=df_Sold_Inflation_new[df_Sold_Inflation_new['Zip']=='33186']
df_NY_Orig=df_Sold_Inflation_new[df_Sold_Inflation_new['Zip']=='11368']
df_OR_Orig=df_Sold_Inflation_new[df_Sold_Inflation_new['Zip']=='97229']

df_Chi_Orig

,Address,City,Zip,SoldDate,Bathrooms,Bedrooms,LivingArea,LandArea,PropPrice,InflationRate,FederalInt,MonthID
137,3839 W 65th St,Chicago,60629,2/14/24,1.0,2,1053,4092.00,225000,3.10,5.33,2
138,5830 S Tripp Ave,Chicago,60629,2/12/24,3.0,6,1356,3750.00,249900,3.10,5.33,2
139,3708 W 56th Pl,Chicago,60629,2/9/24,2.0,4,934,4356.00,309000,3.10,5.33,2
140,6847 S Kolin Ave,Chicago,60629,2/9/24,1.0,3,1090,4687.06,270000,3.10,5.33,2
141,3854 W 59th St,Chicago,60629,2/7/24,3.0,3,1156,3124.99,318000,3.10,5.33,2
...,...,...,...,...,...,...,...,...,...,...,...,...
731,6222 S California Ave,Chicago,60629,8/19/22,1.0,3,950,4375.00,192500,8.26,2.33,8
732,3814 W 69th St,Chicago,60629,8/19/22,1.0,2,864,4125.00,20000,8.26,2.33,8
733,6442 S Francisco Ave,Chicago,60629,8/19/22,1.5,4,1215,3659.00,130000,8.26,2.33,8
734,5514 S Sacramento Ave,Chicago,60629,8/18/22,3.0,7,3200,4142.56,359000,8.26,2.33,8


In [44]:
#extracting data for the Goal Seek model (Prop Price)
column_data = df_Chi_Orig['PropPrice']

# Compute the statistics
min_value = column_data.min()
max_value = column_data.max()
average = column_data.mean()
std_deviation = column_data.std()

# Print the results
print(f"Minimum: {min_value}")
print(f"Maximum: {max_value}")
print(f"Average: {average}")
print(f"Standard Deviation: {std_deviation}")

Minimum: 5000
Maximum: 494000
Average: 238716.1652754591
Standard Deviation: 71856.19752247557


In [47]:
#extracting data for the Goal Seek model (Bedrooms)
column_data_bdrm = df_Chi_Orig['Bedrooms']

# Compute the statistics
min_value_br = column_data_bdrm.min()
max_value_br = column_data_bdrm.max()


# Print the results
print(f"Minimum: {min_value_br}")
print(f"Maximum: {max_value_br}")


Minimum: 1
Maximum: 8


In [48]:
#extracting data for the Goal Seek model (LandArea)
column_data_larea = df_Chi_Orig['LandArea']

# Compute the statistics
min_value_Larea = column_data_larea.min()
max_value_Larea = column_data_larea.max()


# Print the results
print(f"Minimum: {min_value_Larea}")
print(f"Maximum: {max_value_Larea}")


Minimum: 450.0
Maximum: 18295.2


In [50]:
#extracting data for the Goal Seek model (LivingArea)
column_data_livarea = df_Chi_Orig['LivingArea']

# Compute the statistics
min_value_Livarea = column_data_livarea.min()
max_value_Livarea = column_data_livarea.max()


# Print the results
print(f"Minimum: {min_value_Livarea}")
print(f"Maximum: {max_value_Livarea}")


Minimum: 400
Maximum: 6250


In [49]:
#extracting data for the Goal Seek model (Bathrooms)
column_data_ba = df_Chi_Orig['Bathrooms']

# Compute the statistics
min_value_ba = column_data_ba.min()
max_value_ba = column_data_ba.max()


# Print the results
print(f"Minimum: {min_value_ba}")
print(f"Maximum: {max_value_ba}")


Minimum: 1.0
Maximum: 5.0


In [27]:
# Import the PCA module
# Import the modules
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [28]:
# Instantiate the PCA instance and declare the number of PCA variables
pca=PCA(n_components=3)

In [29]:
# Fit the PCA model on the transformed chicago DataFrame
propertyChi_pca = pca.fit_transform(df_Chi)

# Review the first 5 rows of the array of list data
propertyChi_pca[:5]

array([[-1.19388819, -0.07295396,  0.05090529],
       [ 1.45498645, -0.00238342, -0.24876846],
       [ 0.11833415, -0.07581716, -0.18323025],
       [-0.59141193, -0.25146111,  0.05939779],
       [-0.35638754,  0.52121062, -0.30152327]])

In [30]:
# Calculate the PCA explained variance ratio
sum(pca.explained_variance_ratio_), pca.explained_variance_ratio_

(0.9969825548075653, array([0.85099756, 0.09118478, 0.05480021]))

In [31]:
# Create the PCA DataFrame
propertyChi_pca_df = pd.DataFrame(
    propertyChi_pca,
    columns=["PCA1", "PCA2","PCA3"]
)

# Review the PCA DataFrame
propertyChi_pca_df.head()

,PCA1,PCA2,PCA3
0,-1.193888,-0.072954,0.050905
1,1.454986,-0.002383,-0.248768
2,0.118334,-0.075817,-0.183230
3,-0.591412,-0.251461,0.059398
4,-0.356388,0.521211,-0.301523


In [32]:
# Create a a list to store inertia values and the values of k
inertia = []
k = list(range(1, 11))

In [33]:
# Create a for-loop where each value of k is evaluated using the K-means algorithm
# Fit the model using the service_ratings DataFrame
# Append the value of the computed inertia from the `inertia_` attribute of the KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=0)
    k_model.fit(propertyChi_pca_df)
    inertia.append(k_model.inertia_)

In [34]:
# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}

# Create the DataFrame from the elbow data
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

,k,inertia
0,1,470.865692
1,2,204.771479
2,3,131.532185
3,4,104.982239
4,5,79.394835


In [35]:
# Plot the DataFrame
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

In [36]:
# Define the model Kmeans model using the optimal value of k for the number of clusters.
model = KMeans(n_clusters=3, random_state=0)

# Fit the model
model.fit(propertyChi_pca_df)

# Make predictions
k_3= model.predict(propertyChi_pca_df)

# Create a copy of the customers_pca_df DataFrame
propertyChi_pca_predictions_df = propertyChi_pca_df.copy()

# Add a class column with the labels
propertyChi_pca_predictions_df["property_segments"] = k_3

In [39]:
# Plot the clusters
# propertyChi_pca_predictions_df.hvplot.scatter(
#     x="PCA1",
#     y="PCA2",
#     by="property_segments"
# )

import plotly.express as px

fig = px.scatter_3d(propertyChi_pca_predictions_df, x='PCA1', y='PCA2', z='PCA3',
              color='property_segments')
fig.show()

In [40]:
# Define the model Kmeans model using k=3 clusters
model = KMeans(n_clusters=3, random_state=0)

# Fit the model
model.fit(df_Chi)

# Make predictions
k_3 = model.predict(df_Chi)

# Create a copy of the customers_transformed_df DataFrame
df_Chi_predictions = df_Chi.copy()
df_Chi_Orig_Pred= df_Chi_Orig.copy()
# Add a class column with the labels
df_Chi_predictions["property_segments"] = k_3
df_Chi_Orig_Pred["property_segments"] = k_3
df_Chi_Orig_Pred

,Address,City,Zip,SoldDate,Bathrooms,Bedrooms,LivingArea,LandArea,PropPrice,InflationRate,FederalInt,MonthID,property_segments
137,3839 W 65th St,Chicago,60629,2/14/24,1.0,2,1053,4092.00,225000,3.10,5.33,2,0
138,5830 S Tripp Ave,Chicago,60629,2/12/24,3.0,6,1356,3750.00,249900,3.10,5.33,2,2
139,3708 W 56th Pl,Chicago,60629,2/9/24,2.0,4,934,4356.00,309000,3.10,5.33,2,1
140,6847 S Kolin Ave,Chicago,60629,2/9/24,1.0,3,1090,4687.06,270000,3.10,5.33,2,0
141,3854 W 59th St,Chicago,60629,2/7/24,3.0,3,1156,3124.99,318000,3.10,5.33,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,6222 S California Ave,Chicago,60629,8/19/22,1.0,3,950,4375.00,192500,8.26,2.33,8,0
732,3814 W 69th St,Chicago,60629,8/19/22,1.0,2,864,4125.00,20000,8.26,2.33,8,0
733,6442 S Francisco Ave,Chicago,60629,8/19/22,1.5,4,1215,3659.00,130000,8.26,2.33,8,1
734,5514 S Sacramento Ave,Chicago,60629,8/18/22,3.0,7,3200,4142.56,359000,8.26,2.33,8,2


In [41]:
# Plot the clusters per 2 features from the LA dataframe against property price
df_Chi_Orig_Pred.hvplot.scatter(
    x="Bedrooms",
    y="PropPrice",
    by="property_segments",
    hover_cols=['Address', 'SoldDate']
)
#conclusion for this visual: Chicago homeowners are pretty price agnostic. They are purchasing based on number of bedrooms and home renovation status. 3out of the 4 pricier outliers were on the same street, newly constructed in 2022

:NdOverlay   [property_segments]
   :Scatter   [Bedrooms]   (PropPrice,Address,SoldDate)

In [42]:
import plotly.express as px

fig = px.scatter_3d(df_Chi_Orig_Pred, x='Bedrooms', y='PropPrice', z='LandArea',
              color='property_segments')
fig.show()